# ADS 509 Sentiment Assignment

This notebook holds the Sentiment Assignment for Module 6 in ADS 509, Applied Text Mining. Work through this notebook, writing code and answering questions where required. 

In a previous assignment you put together Twitter data and lyrics data on two artists. In this assignment we apply sentiment analysis to those data sets. If, for some reason, you did not complete that previous assignment, data to use for this assignment can be found in the assignment materials section of Blackboard. 


In [1]:
import os
import re
import emoji
import pandas as pd
import numpy as np

from collections import Counter, defaultdict
from string import punctuation

from nltk.corpus import stopwords

sw = stopwords.words("english")

In [109]:
# Add any additional import statements you need here
import warnings
import seaborn

In [19]:
# change `data_location` to the location of the folder on your machine.
data_location = "/Users/clairebentzen/Desktop/MDAS/ADS 509 - Applied Text Mining/Module 2/Assignment2.1/M1 Results/"

# These subfolders should still work if you correctly stored the 
# data from the Module 1 assignment
twitter_folder = "twitter/"
lyrics_folder = "lyrics/"

# Specify artist_files for twitter data
artist_files = {'cher':'cher_followers_data.txt',
                'robyn':'robynkonichiwa_followers_data.txt'}

positive_words_file = "positive-words.txt"
negative_words_file = "negative-words.txt"
tidy_text_file = "tidytext_sentiments.txt"

## Data Input

Now read in each of the corpora. For the lyrics data, it may be convenient to store the entire contents of the file to make it easier to inspect the titles individually, as you'll do in the last part of the assignment. In the solution, I stored the lyrics data in a dictionary with two dimensions of keys: artist and song. The value was the file contents. A Pandas data frame would work equally well. 

For the Twitter data, we only need the description field for this assignment. Feel free all the descriptions read it into a data structure. In the solution, I stored the descriptions as a dictionary of lists, with the key being the artist. 




In [11]:
# Read in the lyrics data
# Specify pathway to lyrics folder
lyrics_path = data_location + lyrics_folder

# Create a dataframe to store results
lyrics_data = pd.DataFrame(columns=['artist', 'song', 'lyrics'])

# Iterate through each file in the lyrics folder
for artist in os.listdir(lyrics_path):
    artist_path = os.path.join(lyrics_path, artist)
    
    # Iterate through each file in the artist folders
    for song in os.listdir(artist_path):
        song_path = os.path.join(artist_path, song)
        rem_prefix = song.removeprefix(f'{artist}_')
        song_title = rem_prefix.removesuffix('.txt')

        # Open and read the contents of the file (song)
        with open(song_path, 'r') as file:
            contents = file.read()
            # Prepare data to add to dataframe
            data = {'artist': artist, 'song': song_title, 'lyrics': contents}
            # The df.append() function is deprecated, so we will ignore warnings here
            with warnings.catch_warnings():
                warnings.simplefilter('ignore')
                # Append row of data to lyrics_df
                lyrics_data = lyrics_data.append(data, ignore_index=True)

In [15]:
# View lyrics_data dataframe
lyrics_data.head()

,artist,song,lyrics
0,robyn,includemeout,"""Include Me Out""\n\n\n\nIt is really very simp..."
1,robyn,electric,"""Electric""\n\n\n\nElectric...\n\nIt's electric..."
2,robyn,beach2k20,"""Beach 2K20""\n\n\n\n(So you wanna go out?\nHow..."
3,robyn,lovekills,"""Love Kills""\n\n\n\nIf you're looking for love..."
4,robyn,timemachine,"""Time Machine""\n\n\n\nHey, what did I do?\nCan..."


In [21]:
# Read in the twitter data
# Read cher twitter data
twitter_data = pd.read_csv(data_location + twitter_folder + artist_files['cher'],
                           sep="\t",
                           quoting=3)

twitter_data['artist'] = "cher"

# Read robyn twitter data
twitter_data_2 = pd.read_csv(data_location + twitter_folder + artist_files['robyn'],
                             sep="\t",
                             quoting=3)
twitter_data_2['artist'] = "robyn"

# Concat twitter dataframes
twitter_data = pd.concat([
    twitter_data,twitter_data_2])
    
del(twitter_data_2)

In [25]:
# Keep only artist and description columns
twitter_data = twitter_data[['description', 'artist']]

In [69]:
# Read in the positive and negative words and the
# tidytext sentiment. Store these so that the positive
# words are associated with a score of +1 and negative words
# are associated with a score of -1. You can use a dataframe or a 
# dictionary for this.

# Read positive words and add score column
pos_words = pd.read_csv('positive-words.txt', delimiter='\t', skiprows=35, header=None)
pos_words['score'] = 1
pos_words.columns.values[0] = 'word'

# Read negative words and add score column
neg_words = pd.read_csv('negative-words.txt', delimiter='\t', skiprows=35, header=None)
neg_words['score'] = -1
neg_words.columns.values[0] = 'word'

# Read tidytext sentiment
tidytext = pd.read_csv('tidytext_sentiments.txt', delimiter='\t')
tidytext.drop(columns='lexicon', inplace=True)
# Replace sentiment with -1 and 1
tidytext['score'] = tidytext['sentiment'].apply(lambda x: -1 if x == 'negative' else 1)
tidytext.drop(columns='sentiment', inplace=True)

# Combine dataframes
words = pd.concat([pos_words, neg_words, tidytext])
words

,word,score
0,a+,1
1,abound,1
2,abounds,1
3,abundance,1
4,abundant,1
...,...,...
15128,win,1
15129,winner,1
15130,winners,1
15131,winning,1


In [27]:
# View twitter_data dataframe
twitter_data.head()

,description,artist
0,NaN,cher
1,𝙿𝚛𝚘𝚞𝚍 𝚜𝚞𝚙𝚙𝚘𝚛𝚝𝚎𝚛 𝚘𝚏 𝚖𝚎𝚜𝚜𝚢 𝚋𝚞𝚗𝚜 & 𝚕𝚎𝚐𝚐𝚒𝚗𝚐𝚜,cher
2,163㎝／愛かっぷ💜26歳🍒 工〇好きな女の子💓 フォローしてくれたらDMします🧡,cher
3,csu,cher
4,Writer @Washinformer @SpelmanCollege alumna #D...,cher


## Sentiment Analysis on Songs

In this section, score the sentiment for all the songs for both artists in your data set. Score the sentiment by manually calculating the sentiment using the combined lexicons provided in this repository. 

After you have calculated these sentiments, answer the questions at the end of this section.


In [71]:
# Punctuation
punctuation = set(punctuation) # speeds up comparison

# Stopwords
sw = stopwords.words("english")

# Removes stopwords
def remove_stop(tokens):
    
    tokens = [word for word in tokens if word not in sw]
    
    return(tokens)
 
# Removes punctuation
def remove_punctuation(text, punct_set=punctuation): 
    
    return("".join([ch for ch in text if ch not in punct_set]))

# Tokenizes the test
def tokenize(text):     
    
    return text.split()

# Create function to calculate sentiment score
def calculate_score(text): 

    # Lower case
    text = str.lower(text)

    # Remove punctuation
    text = remove_punctuation(text)

    # Tokenize text
    tokens = tokenize(text)

    # Remove stopwords
    tokens = remove_stop(tokens)

    # 
    sentiment_score = 0
    
    for word in tokens:
        if word in words['word'].values:
            score = words.loc[words['word'] == word, 'score'].values[0]
            sentiment_score += score
            
    return sentiment_score / len(tokens)

In [75]:
# 
lyrics_data['sentiment_score'] = lyrics_data['lyrics'].apply(calculate_score)

In [81]:
lyrics_avg = lyrics_data.groupby('artist')['sentiment_score'].mean()
lyrics_avg

artist
cher     0.051282
robyn    0.062239
Name: sentiment_score, dtype: float64

In [103]:
#
cher_lyrics = lyrics_data[lyrics_data['artist'] == 'cher']

# Get the top 3 scores
cher_top_3 = cher_lyrics.nlargest(3, 'sentiment_score')
cher_top_3

,artist,song,lyrics,sentiment_score
119,cher,mylove,"""My Love""\n\n\n\nWhen I go away\nI know my hea...",0.53012
189,cher,sunny,"""Sunny""\n\n\n\nSunny, yesterday my life was fi...",0.46250
250,cher,loveandunderstanding,"""Love And Understanding""\n\n\n\nHere, here in ...",0.38961


In [101]:
# Get the bottom 3 scores
cher_bottom_3 = cher_lyrics.nsmallest(3, 'sentiment_score')
cher_bottom_3

,artist,song,lyrics,sentiment_score
145,cher,bangbang,"""Bang-Bang""\n\n\n\nBang bang you shot me down\...",-0.408046
299,cher,bangbangmybabyshotmedown,"""Bang Bang (My Baby Shot Me Down)""\n\n\n\nI wa...",-0.350000
344,cher,crymyselftosleep,"""Cry Myself To Sleep""\n\n\n\nEvery night, I la...",-0.258065


In [105]:
#
robyn_lyrics = lyrics_data[lyrics_data['artist'] == 'robyn']

# Get the top 3 scores
robyn_top_3 = robyn_lyrics.nlargest(3, 'sentiment_score')
robyn_top_3

,artist,song,lyrics,sentiment_score
64,robyn,babyforgiveme,"""Baby Forgive Me""\n\n\n\nHere come the night\n...",0.532468
21,robyn,loveisfree,"""Love Is Free""\n\n\n\nFree\nLove is free, baby...",0.403175
49,robyn,betweenthelines,"""Between The Lines""\n\n\n\nEven reading in bet...",0.313253


In [107]:
# Get the bottom 3 scores
robyn_bottom_3 = robyn_lyrics.nsmallest(3, 'sentiment_score')
robyn_bottom_3

,artist,song,lyrics,sentiment_score
53,robyn,dontfuckingtellmewhattodo,"""Don't Fucking Tell Me What To Do""\n\n\n\nMy d...",-0.514124
75,robyn,dontfuckingtellmewhattodo114520,"""Don't Fucking Tell Me What To Do""\n\n\n\nMy d...",-0.514124
16,robyn,criminalintent,"""Criminal Intent""\n\n\n\nSomebody alert the au...",-0.183746


### Questions

Q: Overall, which artist has the higher average sentiment per song? 

A: <!-- Your answer here -->

---

Q: For your first artist, what are the three songs that have the highest and lowest sentiments? Print the lyrics of those songs to the screen. What do you think is driving the sentiment score? 

A: <!-- Your answer here -->

---

Q: For your second artist, what are the three songs that have the highest and lowest sentiments? Print the lyrics of those songs to the screen. What do you think is driving the sentiment score? 

A: <!-- Your answer here -->

---

Q: Plot the distributions of the sentiment scores for both artists. You can use `seaborn` to plot densities or plot histograms in matplotlib.




## Sentiment Analysis on Twitter Descriptions

In this section, define two sets of emojis you designate as positive and negative. Make sure to have at least 10 emojis per set. You can learn about the most popular emojis on Twitter at [the emojitracker](https://emojitracker.com/). 

Associate your positive emojis with a score of +1, negative with -1. Score the average sentiment of your two artists based on the Twitter descriptions of their followers. The average sentiment can just be the total score divided by number of followers. You do not need to calculate sentiment on non-emoji content for this section.

In [111]:
# your code here
data = {'emoji': ['😂', '❤️', '😭', '😍', '♻️', '♥️', '😊', '😔', '💕', '😘', 
                  '😒', '😩', '😁', '🔥', '💔', '😢', '😳', '😡', '😞', '😑'],
        'score': [1, 1, -1, 1, 1, 1, 1, -1, 1, 1, 
                  -1, -1, 1, 1, -1, -1, -1, -1, -1, -1]}
emojis = pd.DataFrame(data)

In [113]:
# Checks if contains emojis
def contains_emoji(s):
    
    s = str(s)
    emojis = [ch for ch in s if emoji.is_emoji(ch)]

    return(len(emojis) > 0)

# Check which twitter descriptions have emojis
twitter_data['has_emoji'] = twitter_data["description"].apply(contains_emoji)

In [119]:
twitter_data = twitter_data[twitter_data['has_emoji']==True]


,description,artist,has_emoji
2,163㎝／愛かっぷ💜26歳🍒 工〇好きな女の子💓 フォローしてくれたらDMします🧡,cher,True
10,do what thou wilt. 🖤✨,cher,True
22,Ubuntu to Mudita 🧩 INFP🌻,cher,True
25,Romanos 8:31 ✝️,cher,True
31,Just some guy 🙃,cher,True
...,...,...,...
358344,"Artist and promoter, NHS worker, progressive r...",robyn,True
358347,"A beer-drinking, sport-watching, c̶a̶t̶-̶o̶w̶n...",robyn,True
358359,"""Fuck tragedy. Love, Gracey"" #AllBlackLivesMat...",robyn,True
358360,"🏀 Lakers loving, ⚾ Dodger blue bleeding Produc...",robyn,True


Q: What is the average sentiment of your two artists? 

A: <!-- Your answer here --> 

---

Q: Which positive emoji is the most popular for each artist? Which negative emoji? 

A: <!-- Your answer here --> 

